In [1]:
import QuantLib as ql
import math

In [2]:
today = ql.Date(27, ql.July, 2018)
ql.Settings.instance().evaluationDate = today
calendar = ql.UnitedStates(ql.UnitedStates.NYSE)

In [3]:
exercise_date = today + ql.Period(3, ql.Months)
strike = 100.0
option = ql.EuropeanOption(ql.PlainVanillaPayoff(ql.Option.Call, strike), ql.EuropeanExercise(exercise_date))

In [4]:
u = ql.RelinkableQuoteHandle()
r = ql.RelinkableYieldTermStructureHandle()
sigma = ql.RelinkableBlackVolTermStructureHandle()

In [5]:
process = ql.BlackScholesProcess(u, r, sigma)

In [6]:
analytic_engine = ql.AnalyticEuropeanEngine(process)
fd_engine = ql.FDEuropeanEngine(process, 1000, 1000)

In [7]:
u.linkTo(ql.SimpleQuote(100.0))
r.linkTo(ql.FlatForward(today, 0.0, ql.Actual365Fixed()))
sigma.linkTo(ql.BlackConstantVol(today, calendar, 0.20, ql.Actual365Fixed()))

In [8]:
option.setPricingEngine(analytic_engine)
option.NPV()

4.004101982740124

In [9]:
option.setPricingEngine(fd_engine)
option.NPV()

4.004154055896805

In [10]:
sigma.linkTo(ql.BlackConstantVol(today, calendar, 0.20, ql.Business252(calendar)))

In [11]:
option.setPricingEngine(analytic_engine)
option.NPV()

4.050510859367279

In [12]:
option.setPricingEngine(fd_engine)
option.NPV()

4.004154055896805

In [13]:
vol = sigma.blackVol(exercise_date, strike)
vol

0.2

In [14]:
T_vol = sigma.dayCounter().yearFraction(today, exercise_date)
T_vol

0.25793650793650796

In [15]:
T_grid = r.dayCounter().yearFraction(today, exercise_date)
T_grid

0.25205479452054796

In [16]:
vol*vol*T_grid

0.01008219178082192

In [17]:
var = sigma.blackVariance(exercise_date, strike)
var

0.01031746031746032

In [18]:
vol = math.sqrt(var/T_grid)
vol

0.20232004929429467

In [19]:
sigma.linkTo(ql.BlackConstantVol(today, calendar, vol, ql.Actual365Fixed()))

In [20]:
option.setPricingEngine(analytic_engine)
option.NPV()

4.050510859367279

In [21]:
option.setPricingEngine(fd_engine)
option.NPV()

4.050563403337715